## Why is My Bus Always Late?
Our original question comes from the experience of regular bus rider wondering why their bus is always late. We use data to study patterns in the actual arrival times of buses at a stop, compared to their scheduled times. This information will help us better understand what it's like to wait for the bus.

We narrow our investigation to one bus stop in the city of Seattle. ```The data we have are for the stops of Seattle's Rapid Ride lines C, D, and E at Third Avenue and Pike Street.```

* Data Provider: The Washington State Transportation Center
* Data Period: March 26 and May 27, 2016

In [49]:
import pandas as pd
import numpy as np
import plotly.express as px

In [5]:
bus.tail()

,OPD_DATE,VEHICLE_ID,RTE,DIR,TRIP_ID,STOP_ID,STOP_NAME,SCH_STOP_TM,ACT_STOP_TM
39152,2016-05-27,6215,674,N,30905957,578,3RD AVE & PIKE ST (578),10:50:25,10:49:50
39153,2016-05-27,6215,674,N,30905720,578,3RD AVE & PIKE ST (578),13:02:27,13:07:10
39154,2016-05-27,6215,674,N,30905729,578,3RD AVE & PIKE ST (578),15:14:29,15:15:55
39155,2016-05-27,6215,674,N,30905740,578,3RD AVE & PIKE ST (578),17:26:29,17:25:40
39156,2016-05-27,6216,674,N,30905885,578,3RD AVE & PIKE ST (578),18:06:29,18:07:01


It looks like some of the columns in the table might be redundant, like the columns labeled ```STOP_ID``` and ```STOP_NAME```. We can find the number of unique values and their counts to confirm this:

In [6]:
bus[['STOP_ID', 'STOP_NAME']].value_counts()

STOP_ID  STOP_NAME              
578      3RD AVE & PIKE ST (578)    19686
431      3RD AVE & PIKE ST (431)    19471
Name: count, dtype: int64

There are two ```3RD AVE & PIKE ST``` names for the stop.

In [7]:
bus[['DIR', 'STOP_ID', 'STOP_NAME']].value_counts()

DIR  STOP_ID  STOP_NAME              
N    578      3RD AVE & PIKE ST (578)    19686
S    431      3RD AVE & PIKE ST (431)    19471
Name: count, dtype: int64

So, the names are related to the direction of the bus. Since we are looking at only one stop in our analysis, we don't really need anything more than the direction.

We can also check the number of unique route names.

In [8]:
bus['RTE'].value_counts()

RTE
673    13358
674    13205
675    12594
Name: count, dtype: int64

#### 5.2. Data Wrangling
We can find the the info about how the RTE numbers match the names C, D, and E from the transit site. So we replace the route numbers with their letters to make the data easier to understand.

In [10]:
def clean_stops(bus):
    return bus.assign(
        route = bus['RTE'].replace({673: "C", 674: "D", 675: "E"}),
        direction = bus['DIR'].replace({"N": "northbound", "S": "southbound"}),
    )

We can use the scheduled and actual arrival times to calculate how late a bus is.

In [24]:
def compute_mins_late(bus):
    bus = bus.assign(
        scheduled = pd.to_datetime(bus['OPD_DATE'] + " " + bus['SCH_STOP_TM']),
        actual = pd.to_datetime(bus['OPD_DATE'] + " " + bus['ACT_STOP_TM']),
    )

    # if scheduled & actual span midnight, then the actual day needs to be adjusted
    minute = pd.Timedelta("1 minute")
    hour = pd.Timedelta("1 hour")

    # When you subtract two datetime objects, you get a Timedelta
    # Dividing a Timedelta by another Timedelta gives you a number
    diff_hrs = (bus['actual'] - bus['scheduled']) / hour

    # e.g. scheduled at 23:50 but arrives at 00:10, the diff_hrs would show -23 hrs (23 hrs early)
    bus.loc[diff_hrs > 20, "actual"] -= 24 * hour # the bus arrives early at the previous day
    bus.loc[diff_hrs < -20, "actual"] += 24 * hour # the bus arrives at the next day
    return bus.assign(
        minutes_late = (bus['actual'] - bus['scheduled']) / minute
    )

bus = (
    pd.read_csv("chapter5/seattle_bus_times.txt")
    .dropna(axis=0, how="any")
    .pipe(clean_stops)
    .pipe(compute_mins_late)
)
bus.head(3)

,OPD_DATE,VEHICLE_ID,RTE,DIR,TRIP_ID,STOP_ID,STOP_NAME,SCH_STOP_TM,ACT_STOP_TM,route,direction,scheduled,actual,minutes_late
0,2016-03-26,6201,673,S,30908177,431,3RD AVE & PIKE ST (431),01:11:57,01:13:19,C,southbound,2016-03-26 01:11:57,2016-03-26 01:13:19,1.366667
1,2016-03-26,6201,673,S,30908033,431,3RD AVE & PIKE ST (431),23:19:57,23:16:13,C,southbound,2016-03-26 23:19:57,2016-03-26 23:16:13,-3.733333
2,2016-03-26,6201,673,S,30908028,431,3RD AVE & PIKE ST (431),21:19:57,21:18:46,C,southbound,2016-03-26 21:19:57,2016-03-26 21:18:46,-1.183333


In [46]:
smallest_amount_late = bus['minutes_late'].min()
greatest_amount_late = bus['minutes_late'].max()
median_amount_late = bus['minutes_late'].median()
print(f"Smallest Amount Late: {smallest_amount_late:.2f} minutes")
print(f"Greatest Amount Late: {greatest_amount_late:.2f} minutes")
print(f"Median Amount Late: {median_amount_late:.2f} minutes")

Smallest Amount Late: -12.87 minutes
Greatest Amount Late: 150.28 minutes
Median Amount Late: 0.52 minutes


Let's take a look at the histogram of how many minutes late the buses are:

In [21]:
px.histogram(bus, x="minutes_late", nbins=120, width=450, height=300,
             labels={"minutes_late": "Minutes late"}).update_layout(bargap=0.1)

Finally, we conclude our wrangling by creating a simplified version of the data table.

In [25]:
bus = bus[["route", "direction", "scheduled", "actual", "minutes_late"]]
bus.head()

,route,direction,scheduled,actual,minutes_late
0,C,southbound,2016-03-26 01:11:57,2016-03-26 01:13:19,1.366667
1,C,southbound,2016-03-26 23:19:57,2016-03-26 23:16:13,-3.733333
2,C,southbound,2016-03-26 21:19:57,2016-03-26 21:18:46,-1.183333
3,C,southbound,2016-03-26 19:04:57,2016-03-26 19:01:49,-3.133333
4,C,southbound,2016-03-26 16:42:57,2016-03-26 16:42:39,-0.300000


#### 5.3. Exploring Bus Times
Before we begin to model wait time, we want to dig deeper to better understand the phenomenon of bus lateness. We narrowed our focus to the bus activity at **one stop (Third Avenue and Pike Street) over a two-month period**. And we saw that the distribution of the lateness of a bus is skewed to the right, with some buses being very late indeed. We might ask:
* Does the distribution of lateness look the same for all three bus lines?
* Does it matter whether the bus is traveling north or south?
* How does the time of day relate to how late the bus is?
* Are the buses scheduled to arrive at regular intervals throughout the day?

In [33]:
fig = px.histogram(bus, x='minutes_late',
                   histnorm='probability density', nbins=200,
                   facet_row='direction', facet_col='route',
                   facet_row_spacing=0.1,
                   labels={'minutes_late': "Minutes late"},
                   width=550, height=400).update_layout(bargap=0.1)

fig.update_xaxes(range=[-12, 25])
fig.update_yaxes(range=[0, 0.3], title="")
# fig.update_layout(margin=dict(t=25))
fig.show()

The scale on the y-axis is proportion (or density). This scale makes it easier to compare the histograms since we are not misled by different counts in the groups. We learn that line C originates in the north and the other two lines originate in the south. ```The histogram imply there is greater variability in arrival times in the second half of the bus routes.``` That makes sense since delays get compounded as the day progresses.

Next, to explore lateness by time of day, we need to derive a new quantity: **the hour of the day that the bus is scheduled to arrive**.

In [35]:
bus['hour_of_day'] = bus['scheduled'].dt.hour

fig = px.box(bus, y='minutes_late', x='hour_of_day',
             facet_row='direction', facet_col='route',
             labels={'minutes_late': "Minutes late",
                     'hour_of_day': 'Hour of day'},
             width=600, height=400).update_layout(bargap=0.1)

fig.update_yaxes(range=[-12, 50])
# fig.update_layout(margin=dict(t=25))
fig.show()

Indeed, there does appear to be a rush-hour effect, and it seems worse for the evening rush hour compared to the morning. The northbound C line looks to be the most impacted.

Lastly, to examine the scheduled frequency of the buses, we need to compute the intervals between scheduled bus times. We create a new column in our table that contains the time between the scheduled arrival times for the northbound C buses.

In [36]:
minute = pd.Timedelta("1 minute")
bus_c_n = (
    bus[(bus['route'] == 'C') & (bus['direction'] == 'northbound')]
    .sort_values('scheduled')
    .assign(sched_inter=lambda x: x['scheduled'].diff() / minute)
)
bus_c_n.head(3)

,route,direction,scheduled,actual,minutes_late,hour_of_day,sched_inter
19512,C,northbound,2016-03-26 00:00:25,2016-03-26 00:05:01,4.600000,0,NaN
19471,C,northbound,2016-03-26 00:30:25,2016-03-26 00:30:19,-0.100000,0,30.0
19487,C,northbound,2016-03-26 01:05:25,2016-03-26 01:10:15,4.833333,1,35.0


Let's examine a histogram of the distribution of inter-arrival times of these buses:

In [39]:
fig = px.histogram(bus_c_n, x='sched_inter', title="Bus line C, northbound",
                   width=450, height=300).update_layout(bargap=0.1)
fig.update_xaxes(range=[0, 40], title="Time between consecutive buses")
fig.update_layout(margin=dict(t=40))

In this two-month period, about 1,500 of the buses are scheduled to arrive 12 minutes apart and about 1,400 are supposed to arrive 15 minutes after the previous bus.

Most notably, if we want to get a clear picture of the experience of waiting for a bus, we need to take into account the scheduled interval between buses, as well as the bus line and direction.

#### 5.4. Modeling Wait Times
We are interested in modeling the experience of someone waiting at a bus stop. We could develop a complex model that involves the intervals between scheduled arrivals, the bus line, and direction. Instead, we take a simpler approach and narrow the focus to one line, one direction, and one scheduled interval.

**We examine the northbound C line stops that are scheduled to arrive 12 minutes apart:**

In [43]:
bus_c_n_12 = bus_c_n[bus_c_n['sched_inter'] == 12]
len(bus_c_n_12)

1434

In [44]:
# Create another histogram of the data that northbound C line stops that are scheduled to arrive 12 minutes after the previous bus:
fig = px.histogram(bus_c_n_12, x='minutes_late',
                   labels={'minutes_late': 'Minutes late'},
                   nbins=120, width=450, height=300).update_layout(bargap=0.1)

fig.add_annotation(x=20, y=150, showarrow=False,
                   text="Line C, northbound<br>Scheduled arrivals: 12 minutes apart")
fig.update_xaxes(range=[-13, 40])
fig.show()

In [48]:
print(f"Smallest amount late: {bus_c_n_12['minutes_late'].min():.2f} minutes")
print(f"Greatest amount late: {bus_c_n_12['minutes_late'].max():.2f} minutes")
print(f"Median amount late: {bus_c_n_12['minutes_late'].median():.2f} minutes")

Smallest amount late: -10.20 minutes
Greatest amount late: 57.00 minutes
Median amount late: -0.50 minutes


Interestingly, the northbound buses on the C line that are 12 minutes apart are more often early than not!

Let's revisit our question to confirm that we are on track for answering it. ```A summary of how late the buses are does not quite address the experience of the person waiting for the bus. ```When someone arrives at a bus stop, they need to wait for the next bus to arrive. **If people are arriving at the bust stop at random times, notice that they are more likely to arrive in a time interval where the bus is delayed because there's a longer interval between buses.** This arrival pattern is an example of size-biased sampling. So to answer the question of what do people experience when waiting for a bus, we need to do more than summarize how late the bus is.

We can design a simulation that mimics waiting for a bus over the course of one day. To do this, **we set up a string of scheduled bus arrivals that are 12 minutes apart from 6 a.m. to midnight**:

In [50]:
scheduled = 12 * np.arange(91)
scheduled

array([   0,   12,   24,   36,   48,   60,   72,   84,   96,  108,  120,
        132,  144,  156,  168,  180,  192,  204,  216,  228,  240,  252,
        264,  276,  288,  300,  312,  324,  336,  348,  360,  372,  384,
        396,  408,  420,  432,  444,  456,  468,  480,  492,  504,  516,
        528,  540,  552,  564,  576,  588,  600,  612,  624,  636,  648,
        660,  672,  684,  696,  708,  720,  732,  744,  756,  768,  780,
        792,  804,  816,  828,  840,  852,  864,  876,  888,  900,  912,
        924,  936,  948,  960,  972,  984,  996, 1008, 1020, 1032, 1044,
       1056, 1068, 1080])

Based on the scheduled arrival times we get above, we simulate the actual arrival times by adding a random number of minutes each bus is late. To do this, **we choose the minutes late from the distribution of observed lateness of the actual buses (bus_c_n_12["minutes_late"]).

In [51]:
minutes_late = bus_c_n_12['minutes_late']
actual = scheduled + np.random.choice(minutes_late, size=91, replace=True)

We need to sort these actual arrival times because when a bus is super late, another may well come along before it:

In [52]:
actual.sort()
actual

array([  -3.11666667,    7.46666667,   19.5       ,   48.01666667,
         55.46666667,   73.83333333,   74.83333333,   84.13333333,
         94.11666667,  110.93333333,  118.2       ,  130.33333333,
        151.        ,  154.15      ,  166.03333333,  179.66666667,
        189.65      ,  200.8       ,  214.86666667,  235.51666667,
        243.45      ,  250.08333333,  259.13333333,  273.65      ,
        284.63333333,  296.96666667,  306.13333333,  329.36666667,
        336.33333333,  345.63333333,  356.96666667,  374.11666667,
        381.65      ,  396.36666667,  406.15      ,  416.95      ,
        428.66666667,  440.98333333,  454.31666667,  469.83333333,
        476.86666667,  488.83333333,  498.95      ,  512.76666667,
        522.1       ,  544.73333333,  551.61666667,  560.95      ,
        573.55      ,  585.83333333,  598.33333333,  611.2       ,
        624.78333333,  637.43333333,  647.36666667,  672.33333333,
        679.38333333,  682.18333333,  691.63333333,  702.95   

We also need to simulate the arrival of people at the bus stop at random times throughout the day. For the passengers, we put a marble in the urn with a time on it. These run from time 0, which stands for 6 a.m., to the arrival of the bus at midnight, which is 1,068 minutes past 6 a.m.. To match the way the bus times are measured in our data, we make the times 1/100 of a minute apart:

In [53]:
pass_arrival_times = np.arange(100*1068)
# Being divided by 100 to show the final arrival times we need, but this step didn't change the values
pass_arrival_times / 100

array([0.00000e+00, 1.00000e-02, 2.00000e-02, ..., 1.06797e+03,
       1.06798e+03, 1.06799e+03], shape=(106800,))

Now we can simulate the arrival of, say, 500 people at the bus stop throughout the day. We draw 500 times from this urn, replacing the marbles between draws:

In [54]:
sim_arrival_times = (
    np.random.choice(pass_arrival_times, size=500, replace=True) / 100
)
sim_arrival_times.sort()
sim_arrival_times

array([   1.18,    5.96,    8.09,   11.71,   15.37,   15.43,   16.  ,
         17.3 ,   19.49,   21.89,   23.16,   23.57,   24.32,   25.42,
         25.62,   25.98,   26.58,   26.93,   27.02,   31.1 ,   32.05,
         32.62,   34.46,   35.21,   41.66,   41.74,   43.73,   44.4 ,
         47.07,   57.27,   57.46,   61.74,   66.02,   68.42,   68.49,
         71.  ,   73.91,   74.01,   75.48,   76.7 ,   81.38,   86.53,
         88.38,   89.  ,   90.5 ,   90.58,   91.79,   93.04,   93.32,
         97.11,   97.91,  100.08,  100.54,  101.44,  104.8 ,  105.44,
        107.85,  124.48,  126.1 ,  127.1 ,  132.07,  132.6 ,  133.3 ,
        133.69,  133.97,  136.54,  139.97,  140.18,  143.86,  145.24,
        149.2 ,  150.52,  150.71,  152.09,  153.85,  157.41,  161.1 ,
        161.65,  168.74,  169.31,  169.73,  179.93,  180.11,  182.46,
        183.91,  183.95,  184.62,  187.16,  190.14,  194.48,  195.  ,
        197.  ,  199.84,  202.7 ,  204.84,  214.29,  215.22,  218.27,
        218.92,  220

To find out how long each individual waits, we look for the soonest bus to arrive after their sampled time.

In [57]:
i = np.searchsorted(actual, sim_arrival_times, side='right')
sim_wait_times = actual[i] - sim_arrival_times
sim_wait_times

array([6.28666667e+00, 1.50666667e+00, 1.14100000e+01, 7.79000000e+00,
       4.13000000e+00, 4.07000000e+00, 3.50000000e+00, 2.20000000e+00,
       1.00000000e-02, 2.61266667e+01, 2.48566667e+01, 2.44466667e+01,
       2.36966667e+01, 2.25966667e+01, 2.23966667e+01, 2.20366667e+01,
       2.14366667e+01, 2.10866667e+01, 2.09966667e+01, 1.69166667e+01,
       1.59666667e+01, 1.53966667e+01, 1.35566667e+01, 1.28066667e+01,
       6.35666667e+00, 6.27666667e+00, 4.28666667e+00, 3.61666667e+00,
       9.46666667e-01, 1.65633333e+01, 1.63733333e+01, 1.20933333e+01,
       7.81333333e+00, 5.41333333e+00, 5.34333333e+00, 2.83333333e+00,
       9.23333333e-01, 8.23333333e-01, 8.65333333e+00, 7.43333333e+00,
       2.75333333e+00, 7.58666667e+00, 5.73666667e+00, 5.11666667e+00,
       3.61666667e+00, 3.53666667e+00, 2.32666667e+00, 1.07666667e+00,
       7.96666667e-01, 1.38233333e+01, 1.30233333e+01, 1.08533333e+01,
       1.03933333e+01, 9.49333333e+00, 6.13333333e+00, 5.49333333e+00,
      

We can set up a complete simulation where we simulate, say, 200 days of bus arrivals, and for each day, we simulate 500 people arriving at the bus stop at random times throughout the day. In total, that's 100,000 simulated wait times:

In [59]:
sim_wait_times = []

for day in np.arange(0, 200, 1):
    bus_late = np.random.choice(minutes_late, size=91, replace=True)
    actual = scheduled + bus_late
    actual.sort()
    sim_arrival_times = (
        np.random.choice(pass_arrival_times, size=500, replace=True) / 100
    )
    sim_arrival_times.sort()
    i = np.searchsorted(actual, sim_arrival_times, side='right')
    sim_wait_times = np.append(sim_wait_times, actual[i] - sim_arrival_times)

Let's make a histogram of these simulated wait times to examine the distribution:

In [64]:
fig = px.histogram(x=sim_wait_times, nbins=40,
                   histnorm='probability density', width=450, height=300).update_layout(bargap=0.1)

fig.update_xaxes(title="Simulated wait times for 100,000 passengers")
fig.update_yaxes(title="proportion")
fig.show()

As we expect, we find a skewed distribution. We can model this by a constant model where we use absolute loss to select the best constant. The absolute loss gives us the median wait time:

In [66]:
print(f"Median wait time: {np.median(sim_wait_times):.2f} minutes")

Median wait time: 6.49 minutes


We can compute the upper quartile of wait times to give us a sense of variability:

In [68]:
# np.quantile() finds the value that splits the data into the bottom 75% and top 25%
print(f"Upper quartile: {np.quantile(sim_wait_times, 0.75):.2f} minutes")

Upper quartile: 10.61 minutes


It's undoubtedly memorable when you have to wait more than 10 minutes for a bus that is supposed to arrive every 12 minutes, and this happens one in four times you take the bus!